In [1]:
import abc
from abc import abstractmethod
from collections import Sequence

<ipython-input-1-ccedda125738>:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Sequence


In [2]:
class EmptyException(Exception):
    pass

class LinkedStack:

    class _Node:

        def __init__(self, element, next_node):
            self.element = element
            self.next = next_node

    def __init__(self):
        self.head = None
        self.size = 0

    def __len__(self):
        return self.size

    def is_empty(self):
        return self.size == 0

    def push(self, e):
        self.head = self._Node(e, self.head)
        self.size +=1

    def top(self):
        if self.is_empty():
            raise EmptyException("Empty Stack")
        return self.head.element

    def pop(self):
        if self.is_empty():
            raise EmptyException("Empty Stack")
        elem = self.head.element
        self.head = self.head.next
        self.size -= 1
        return elem

In [3]:
class LinkedQueue:

    class _Node:

        def __init__(self, element, next_node):
            self.element = element
            self.next = next_node

    def __init__(self):
        self.head = None
        self.tail = None
        self.size = 0

    def __len__(self):
        return self.size

    def is_empty(self):
        return self.size == 0

    def first(self):
        if self.is_empty():
            raise EmptyException("Empty Stack")
        return self.head.element

    def enqueue(self, e):
        node = self._Node(e, None)
        if self.is_empty():
            self.head = node
        else:
            self.tail.next = node
        self.tail = node
        self.size += 1

    def dequeue(self):
        if self.is_empty():
            raise EmptyException("Empty Stack")
        res = self.head.element
        self.head = self.head.element
        self.size -= 1
        if self.is_empty():
            self.tail = None

        return res

In [4]:
class CircularQueue:

    class _Node:

        def __init__(self, element, next_node):
            self.element = element
            self.next = next_node

    def __init__(self):
        self.tail = None
        self.size = 0

    def __len__(self):
        return self.size

    def is_empty(self):
        return self.size == 0

    def first(self):
        if self.is_empty():
            raise EmptyException("Empty Stack")
        return self.tail.next.element

    def enqueue(self, e):
        node = self._Node(e, None)
        if self.is_empty():
            node.next = node
        else:
            node.next = self.tail.next
            self.tail.next = node
        self.tail = node
        self.size +=1

    def dequeue(self):
        if self.is_empty():
            raise EmptyException("Empty Stack")
        head = self.tail.next
        if self.size == 1:
            self.tail = None
        else:
            self.tail.next = head.next
        self.size -= 1
        return head.element

    def rotate(self):
        if not self.is_empty():
            self.tail = self.tail.next

In [5]:
class DoublyLinkedBase:

    class _Node:
        def __init__(self, element, prev_node, next_node):
            self.element = element
            self.next = next_node
            self.prev_node = prev_node

    def __init__(self):
        self.header = self._Node(None, None, None)
        self.trailer = self._Node(None, None, None)
        self.header.next = self.trailer
        self.trailer.prev = self.header
        self.size = 0

    def __len__(self):
        return self.size

    def is_empty(self):
        return self.size == 0

    def insert_between(self, e, predecessor, successor):
        node = self._Node(e, predecessor, successor)
        predecessor.next = node
        successor.prev = node
        self.size += 1

        return node

    def delete(self, node):
        if self.size == 0:
            raise ValueError
        node.prev.next = node.next
        node.next.prev = node.prev
        self.size -= 1
        el = node.element
        node.prev = node.next = node.element = None
        return el




In [ ]:
class LinkedDeque(DoublyLinkedBase):

    def first(self):
        if self.is_empty():
            raise EmptyException("Empty Stack")
        return self.header.next.element

    def last(self):
        if self.is_empty():
            raise EmptyException("Empty Stack")
        return self.trailer.prev.element

    def insert_first(self, e):
        self.insert_between(e, self.header, self.header.next)

    def insert_last(self, e):
        self.insert_between(e, self.trailer.prev, self.trailer)

    def delete_first(self):
        if self.is_empty():
            raise EmptyException("Empty Stack")
        self.delete(self.header.next)

    def delete_last(self):
        if self.is_empty():
            raise EmptyException("Empty Stack")
        self.delete(self.trailer.prev)


In [7]:
class PositionalList(DoublyLinkedBase):

    class Position:
        __slots__ = ("container", "node")
        def __init__(self, container, node):
            self.container = container
            self.node = node

        def element(self):
            return self.node.element

        def __eq__(self, other):
            return type(other) is type(self) and self.node is other.node

        def __ne__(self, other):
            return not(self == other)

    def validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError("p must of Position type")
        if p.container is not self:
            raise ValueError("p does not belong to this container")
        if p.node.next is None:
            raise ValueError("p is not longer valid")

        return p.node
    
    def make_position(self, node):
        if node is self.header or node is self.trailer:
            return None
        else:
            return self.Position(self, node)
    
    def first(self):
        return self.make_position(self.header.next)
    
    def last(self):
        return self.make_position(self.trailer.prev)
    
    def before(self, p):
        node = self.validate(p)
        return self.make_position(node.prev)
    
    def after(self, p):
        node = self.validate(p)
        return self.make_position(node.next)
    
    def __iter__(self):
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)

    def insert_between(self, e, predecessor, successor):
        node = super().insert_between(e, predecessor, successor)
        return self.make_position(node)

    def add_first(self, e):
        return self.insert_between(e, self.header, self.header.next)

    def add_last(self, e):
        return self.insert_between(e, self.trailer.prev, self.trailer)

    def add_before(self, p, e):
        node = self.validate(p)
        return self.insert_between(e, node.prev, node)

    def add_after(self, p, e):
        node = self.validate(p)
        return self.insert_between(e, node, node.next)

    def delete(self, p):
        node = self.validate(p)
        return super().delete(node)

    def replace(self, p, e):
        node = self.validate(p)
        old_value = node.element
        node.element = e
        return old_value

    @staticmethod
    def insertion_sort(L):
        if len(L) > 1:
            marker = L.first()
            while marker != L.last():
                pivot = L.after(marker)
                value  = pivot.element()
                if value > marker.pivot:
                    marker = pivot
                else:
                    walk = marker
                    while walk != L.first() and L.before(walk).element() > value:
                        walk = L.before(walk)

                    L.delete(pivot)
                    L.add_before(walk, value)

    def __repr__(self):
        string = []
        for p in self:
            string.append(p.element())
        return ", ".join(string)